In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Load wordlists
from uk_nl__wordlists import nl23_political_words, uk24_political_words

In [3]:
# Fetch dataframes
all_articles = pd.read_parquet('../b__data-collection-with-web-scraping/datasets/news/all_articles_with_id.parquet')

In [4]:
nl_articles = all_articles[all_articles['country'] == 'NL']
uk_articles = all_articles[all_articles['country'] == 'UK']

In [5]:
print(f'The final dataset with articles from both countries is of shape {all_articles.shape[0]}')
print(f"{nl_articles.shape[0]} are from the Netherlands, constituting both NOS ({nl_articles[nl_articles['outlet'] == 'NOS'].shape[0]}) and NU ({nl_articles[nl_articles['outlet'] == 'NU'].shape[0]}) articles")
print(f"{uk_articles.shape[0]} are from the United Kingdom, constituting both BBC ({uk_articles[uk_articles['outlet'] == 'BBC'].shape[0]}) and The Guardian ({uk_articles[uk_articles['outlet'] == 'The Guardian'].shape[0]}) articles")

The final dataset with articles from both countries is of shape 94692
11915 are from the Netherlands, constituting both NOS (6565) and NU (5350) articles
82777 are from the United Kingdom, constituting both BBC (51177) and The Guardian (31600) articles


In [6]:
# Inspect the data
print(f'All NL and UK articles combined constitute a dataset of shape {all_articles.shape}.\n')
print(f'And contains the following columns and datatypes:\n{all_articles.dtypes}')
print('\n')
print(f'For the NL case, that means {nl_articles.shape[0]} articles from NOS ({nl_articles[nl_articles["outlet"]=="NOS"].shape[0]}) and NU ({nl_articles[nl_articles["outlet"]=="NU"].shape[0]}) combined.')
print(f'For the UK case, that means {uk_articles.shape[0]} articles from BBC ({uk_articles[uk_articles["outlet"]=="BBC"].shape[0]}) and NU ({uk_articles[uk_articles["outlet"]=="The Guardian"].shape[0]}) combined.')

All NL and UK articles combined constitute a dataset of shape (94692, 11).

And contains the following columns and datatypes:
country               object
outlet                object
id                    object
url                   object
images                object
datetime      datetime64[ns]
category              object
title                 object
paragraphs            object
alt_txt               object
id_unique             object
dtype: object


For the NL case, that means 11915 articles from NOS (6565) and NU (5350) combined.
For the UK case, that means 82777 articles from BBC (51177) and NU (31600) combined.


Write method to classify political articles

In [7]:
nl_test = nl_articles.sample(n=100, random_state=2)
uk_test = uk_articles.sample(n=100, random_state=2)

In [7]:
def list_occurrences_triple_column_vectorised(dataframe, colnames_of_interest, wordlists):
    df = dataframe.copy().reset_index(drop=True)
    
    for colname in colnames_of_interest:
        df[colname] = df[colname].astype(str)
        
        for identifier, wordlist_options in wordlists.items():
            all_matches = []

            multistring_patterns = [
                (re.compile(rf'\b{re.escape(word.lower())}\b'), word) 
                for word in wordlist_options.get('multistring_matches', [])
            ]
            substring_list = wordlist_options.get('substring_matches', [])
            exact_list = set(wordlist_options.get('exact_matches', []))

            for text in df[colname]:
                matches_for_row = []

                # Multi-strings
                lower_text = text.lower()
                for pattern, canonical_word in multistring_patterns:
                    if pattern.search(lower_text):
                        count = len(pattern.findall(lower_text))
                        matches_for_row.extend([canonical_word] * count)

                # Sub-strings
                if substring_list:
                    tokens = re.findall(r'\w+', text)
                    for token in tokens:
                        if any(sub.lower() in token.lower() for sub in substring_list):
                            matches_for_row.append(token)

                # Exact-matches (case sensitive)
                if exact_list:
                    tokens = re.findall(r'\w+', text)
                    for token in tokens:
                        if token in exact_list:
                            matches_for_row.append(token)

                all_matches.append(matches_for_row)

            df[f'{identifier}__{colname}'] = all_matches

    return df

In [9]:
nl_test_text = list_occurrences_triple_column_vectorised(nl_test, ['title', 'paragraphs', 'alt_txt'], nl23_political_words)
nl_test_text.to_excel('nl_test_df.xlsx')

In [10]:
uk_test_text = list_occurrences_triple_column_vectorised(uk_test, ['title', 'paragraphs', 'alt_txt'], uk24_political_words)
uk_test_text.to_excel('uk_test_df.xlsx')

After a number of validation checks, perform the text analysis on the full text (takes approximately 14 minutes to complete)

In [8]:
nl_text_processed = list_occurrences_triple_column_vectorised(nl_articles, ['title', 'paragraphs', 'alt_txt'], nl23_political_words)
#uk_text_processed = list_occurrences_triple_column_vectorised(uk_articles, ['title', 'paragraphs', 'alt_txt'], uk24_political_words)

In [10]:
nl_text_processed.to_parquet('datasets/NL_articles_text_processed.parquet')
#uk_text_processed.to_parquet('datasets/UK_articles_text_processed.parquet')

Continue from file (correction for The Netherlands only)

In [11]:
nl_articles_text_processed = pd.read_parquet('datasets/NL_articles_text_processed.parquet')

In [12]:
def conditional_remove_and_map_words(input, search_list, mapping):
    '''
    Method that 1) splits the input in modified_input and stripped_input, 2) removes the substrings of the stripped_input
    from the modified_input, and 3) adds a mapping of the stripped_input to the modified_input.

    Example input: ['GroenLinks/PvdA', 'GroenLinks', 'PvdA', 'PvdA', 'Timmermans']
    -- this means that by design of the list_occurrences_double_column method, all substrings of the multistring are duplicates --
    modified_input = ['GroenLinks', 'PvdA', 'PvdA', 'Timmermans]
    stripped_input = ['GroenLinks/PvdA']
    returns: ['PvdA', 'Timmermans', 'GL-PvdA']

    :input: dataframe cell containing a wordlist (to be applied in lambda statement)
    :search_list: list of possible ways to reference a multistring
    :mapping: mapping that should replace any of the references from the search_list with a consistent mapping
    :return: modified_list where substring duplicates are removed and the references to the multi-string are made consistent
    '''

    # Split the input in modified_input and stripped_input, where stripped input contains possible references to a multi-string
    modified_input = [i for i in input if i not in search_list]
    stripped_input = [i for i in input if i in search_list]

    # Loop over the multistring references that appeared in the search_list
    for word in stripped_input:
        # Split the multistring in substrings
        subs = re.split(r'[-/]', word)

        # Remove these substrings from the modified input, since they are duplicates
        for sub in subs:
            if sub in modified_input:
                modified_input.remove(sub)

        # Append a consistent mapping of the multistring
        modified_input.append(mapping)
    return modified_input


In [13]:
glpvda_search_list = ['PvdA-GL', 'GL-PvdA', 'GL/PvdA', 'PvdA/GL', 'GroenLinks/PvdA', 'PvdA/GroenLinks', 'GroenLinks-PvdA', 'PvdA-GroenLinks']
glpvda_mapping = 'GL-PvdA'

for col in ['GL-PvdA__title', 'GL-PvdA__paragraphs', 'GL-PvdA__alt_txt']:
    nl_articles_text_processed[col] = nl_articles_text_processed[col].apply(lambda x: conditional_remove_and_map_words(x, glpvda_search_list, glpvda_mapping))

Write to file

In [14]:
nl_articles_text_processed.to_excel('datasets/nl_articles_text_processed.xlsx')
nl_articles_text_processed.to_parquet('datasets/nl_articles_text_processed.parquet')

In [31]:
nl_articles_text_processed.columns.tolist()

['country',
 'outlet',
 'id',
 'url',
 'images',
 'datetime',
 'category',
 'title',
 'paragraphs',
 'alt_txt',
 'id_unique',
 'PVV__title',
 'GL-PvdA__title',
 'VVD__title',
 'NSC__title',
 'D66__title',
 'BBB__title',
 'CDA__title',
 'SP__title',
 'FVD__title',
 'PvdD__title',
 'CU__title',
 'SGP__title',
 'DENK__title',
 'Volt__title',
 'JA21__title',
 'Bij1__title',
 'BvNL__title',
 'Positions__title',
 'Politics__title',
 'Issues__title',
 'National__title',
 'International__title',
 'PVV__paragraphs',
 'GL-PvdA__paragraphs',
 'VVD__paragraphs',
 'NSC__paragraphs',
 'D66__paragraphs',
 'BBB__paragraphs',
 'CDA__paragraphs',
 'SP__paragraphs',
 'FVD__paragraphs',
 'PvdD__paragraphs',
 'CU__paragraphs',
 'SGP__paragraphs',
 'DENK__paragraphs',
 'Volt__paragraphs',
 'JA21__paragraphs',
 'Bij1__paragraphs',
 'BvNL__paragraphs',
 'Positions__paragraphs',
 'Politics__paragraphs',
 'Issues__paragraphs',
 'National__paragraphs',
 'International__paragraphs',
 'PVV__alt_txt',
 'GL-PvdA__al